In [1]:
from Bio import SeqIO
from collections import Counter
import pandas as pd 
import re
import requests
import json
import time
import glob
import pprint
import seaborn as sns
import pylab as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import pycountry 
import pycountry_convert as pc
pp = pprint.PrettyPrinter()
import numpy as np
from tabulate import tabulate
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import regex as re
import plotly.graph_objects as go
import os

from string import punctuation
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import nltk
from bs4 import BeautifulSoup


 

stemmer=SnowballStemmer("english")
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import networkx as nx
from collections import defaultdict
from itertools import combinations


import textmining
  

%env folder = X:\5_Research\Paul\dsi_origins\

env: folder=X:\5_Research\Paul\dsi_origins\


In [2]:
path = %env folder

### 1. Open and merge data

In [4]:
publist_pmid_int = list(pd.read_csv(path+R'processed_data\clean_pmids.csv', index_col = 0)['PRIMARY_PMID'])

In [5]:
publist_pmcid = list(pd.read_csv(path+R'processed_data\clean_pmcids.csv', index_col = 0)['PRIMARY_PMCID'])

In [6]:
publist_doi = list(pd.read_csv(path+R'processed_data\clean_dois.csv', index_col = 0)['PRIMARY_DOI'])

In [18]:
# start with pmid data
with open(path+R'raw_data\full-data_scholarly-lens_pmid.json') as json_string:
    data_dict_pmid = json.load(json_string)

In [20]:
pubmed_ids = []
abstracts = []
titles = []
keywords = []
dates = []
affiliations = []
affiliations_ids = []
affiliations_countries = []
first_author = []
last_author =  []

for i in data_dict_pmid: 
            for publication in i: 
                nr = 0
                for e in publication['external_ids']:
                    for keys, values in e.items():
                        if values == 'pmid' and int(e['value']) in publist_pmid_int: 
                            nr = nr+1
                            if nr >= 2:
                                break

                            n = 1
                            if 'authors' in publication.keys(): 
                                nr_authors = len(publication['authors'])
                                for author in publication['authors']: # I need the authors, because I need their affiliation countries
                                    if n == 1: 
                                        first_author.append(1)
                                    else: 
                                        first_author.append(0)
                                    if n == nr_authors: 
                                        last_author.append(1)
                                    else: 
                                        last_author.append(0)
                                    n = n+1
                                        
                                    # pubmedid
                                    pubmed_ids.append(int(e['value']))
                                    titles.append(publication['title'])
                                    
                                    abstract_placeholder = ''
                                    if 'abstract' in publication.keys():
                                        abstract_placeholder = publication['abstract']
                                    abstracts.append(abstract_placeholder)
                                    
                                    keyword_placeholder = ''
                                    if 'keywords' in publication.keys():
                                        keyword_placeholder = publication['keywords']
                                    keywords.append(keyword_placeholder)
                                    
                                
                                    
                                    dates.append(publication['year_published'])
                                    
                                    
                                    
                                    #--deleted code for other data'

                                    ##afilliations
                                    affiliation_placeholder = ''
                                    affiliation_ids_placeholder = ''
                                    country_placeholder = np.NaN
                                    if 'affiliations' in author.keys() and author['affiliations'] != []:
                                        affiliation_placeholder = author['affiliations'][0]['name']
                                        
                                        #affiliation ids
                                        if 'ids' in author['affiliations'][0]:
                                            for id_dict in author['affiliations'][0]['ids']:
                                                if 'magid' in id_dict.values():
                                                    affiliation_ids_placeholder = id_dict['value']
                                                break
                                            

                                        #countries
                                        if 'country_code' in author['affiliations'][0].keys():
                                            country_placeholder = author['affiliations'][0]['country_code']

                                    affiliations.append(affiliation_placeholder)
                                    affiliations_ids.append(affiliation_ids_placeholder)
                                    affiliations_countries.append(country_placeholder)
                                    
                                    
                                    
                                

# export it as dictionary
dict_author_edges_pmid = {'pubmed_id': pubmed_ids, 'title': titles, 'abstract': abstracts, 'keywords': keywords, 'year_published': dates, 'first_author': first_author, 'last_author': last_author, 'affiliation_country' : affiliations_countries, 'affiliation': affiliations, 'affiliation_id': affiliations_ids} 
df_scholarly_pmid = pd.DataFrame(dict_author_edges_pmid)

In [21]:
#this is the number of the original The Lens dict
publications_pmid = []
for scroll in data_dict_pmid: 
    for pub in scroll: 
        publications_pmid.append(pub['lens_id'])
len(set(publications_pmid))

81093

In [22]:
#this is the final amount after cleaning pubmeds
len(set(df_scholarly_pmid['pubmed_id']))

81090

In [23]:
#save df scholarly pmid
df_scholarly_pmid.to_csv(path+R'processed_data/scholarly_pmid.csv')

In [24]:
#pmcids

with open(path+R'raw_data\full-data_scholarly-lens_pmcid.json') as json_string:
    data_dict_pmcid = json.load(json_string)

In [25]:
pmcids_clean = [x.lower() for x in list(set(publist_pmcid))]

In [26]:
pmcid_ids = []
titles = []
abstracts = []
keywords = []
dates = []
affiliations = []
affiliations_ids = []
affiliations_countries = []
first_author = []
last_author =  []

for i in data_dict_pmcid: 
            for publication in i: 
                nr = 0
                for e in publication['external_ids']:
                    for keys, values in e.items():
                        if values == 'pmcid' and e['value'] in pmcids_clean: 
                            nr = nr+1
                            if nr >= 2:
                                break

                            n = 1
                            if 'authors' in publication.keys(): 
                                nr_authors = len(publication['authors'])
                                for author in publication['authors']: # I need the authors, because I need their affiliation countries
                                    if n == 1: 
                                        first_author.append(1)
                                    else: 
                                        first_author.append(0)
                                    if n == nr_authors: 
                                        last_author.append(1)
                                    else: 
                                        last_author.append(0)
                                    n = n+1
                                    
                                    # pubmedid
                                    pmcid_ids.append(e['value'])
                                    titles.append(publication['title'])
                                    abstract_placeholder = ''
                                    if 'abstract' in publication.keys():
                                        abstract_placeholder = publication['abstract']
                                    abstracts.append(abstract_placeholder)
                                    
                                    keyword_placeholder = ''
                                    if 'keywords' in publication.keys():
                                        keyword_placeholder = publication['keywords']
                                    keywords.append(keyword_placeholder)
                                    
                                    dates.append(publication['year_published'])
                                    
                                    
                                    #--deleted code for other data'

                                    ##afilliations
                                    affiliation_placeholder = ''
                                    affiliation_ids_placeholder = ''
                                    country_placeholder = np.NaN
                                    if 'affiliations' in author.keys() and author['affiliations'] != [] and author['affiliations'] != [{}]:
                                        affiliation_placeholder = author['affiliations'][0]['name']
                                        
                                        #affiliation ids
                                        if 'ids' in author['affiliations'][0]:
                                            for id_dict in author['affiliations'][0]['ids']:
                                                if 'magid' in id_dict.values():
                                                    affiliation_ids_placeholder = id_dict['value']
                                                break
                                            

                                        #countries
                                        if 'country_code' in author['affiliations'][0].keys():
                                            country_placeholder = author['affiliations'][0]['country_code']

                                    affiliations.append(affiliation_placeholder)
                                    affiliations_ids.append(affiliation_ids_placeholder)
                                    affiliations_countries.append(country_placeholder)
                                    
                                

# export it as dictionary
dict_author_edges_pmcid = {'pmcid_id': pmcid_ids, 'title': titles, 'keywords': keywords, 'abstract': abstracts, 'year_published': dates, 'first_author': first_author, 'last_author': last_author, 'affiliation_country' : affiliations_countries, 'affiliation': affiliations, 'affiliation_id': affiliations_ids} 
df_scholarly_pmcid = pd.DataFrame(dict_author_edges_pmcid)

In [27]:
#nr of publications from the lens dict
publications_pmcid = []
for scroll in data_dict_pmcid: 
    for pub in scroll: 
        publications_pmcid.append(pub['lens_id'])
len(set(publications_pmcid))

17487

In [28]:
#final amount after cleaning
len(set(df_scholarly_pmcid['pmcid_id']))

17481

In [29]:
df_scholarly_pmcid.to_csv(path+R'processed_data\scholarly_pmcid.csv')

In [30]:
#dois

with open(path+R'raw_data\full-data_scholarly-lens_doi.json') as json_string:
    data_dict_doi = json.load(json_string)


In [31]:
doi_ids = []
titles = []
abstracts = []
keywords = []
dates = []
affiliations = []
affiliations_ids = []
affiliations_countries = []
first_author = []
last_author =  []

for i in data_dict_doi: 
            for publication in i: 
                nr = 0
                for e in publication['external_ids']:
                    for keys, values in e.items():
                        if values == 'doi' and e['value'] in publist_doi: 
                            nr = nr+1
                            if nr >= 2:
                                break

                            n = 1
                            if 'authors' in publication.keys(): 
                                nr_authors = len(publication['authors'])
                                for author in publication['authors']: # I need the authors, because I need their affiliation countries
                                    if n == 1: 
                                        first_author.append(1)
                                    else: 
                                        first_author.append(0)
                                    if n == nr_authors: 
                                        last_author.append(1)
                                    else: 
                                        last_author.append(0)
                                    n = n+1
                                    
                                    # pubmedid
                                    doi_ids.append(e['value'])
                                    titles.append(publication['title'])
                                    abstract_placeholder = ''
                                    if 'abstract' in publication.keys():
                                        abstract_placeholder = publication['abstract']
                                    abstracts.append(abstract_placeholder)
                                    
                                    keyword_placeholder = ''
                                    if 'keywords' in publication.keys():
                                        keyword_placeholder = publication['keywords']
                                    keywords.append(keyword_placeholder)
                                    
                                    dates.append(publication['year_published'])
                                    
                                    
                                    #--deleted code for other data'

                                    ##afilliations
                                    affiliation_placeholder = ''
                                    affiliation_ids_placeholder = ''
                                    country_placeholder = np.NaN
                                    if 'affiliations' in author.keys() and author['affiliations'] != [] and author['affiliations'] != [{}]:
                                        affiliation_placeholder = author['affiliations'][0]['name']
                                        
                                        #affiliation ids
                                        if 'ids' in author['affiliations'][0]:
                                            for id_dict in author['affiliations'][0]['ids']:
                                                if 'magid' in id_dict.values():
                                                    affiliation_ids_placeholder = id_dict['value']
                                                break
                                            

                                        #countries
                                        if 'country_code' in author['affiliations'][0].keys():
                                            country_placeholder = author['affiliations'][0]['country_code']

                                    affiliations.append(affiliation_placeholder)
                                    affiliations_ids.append(affiliation_ids_placeholder)
                                    affiliations_countries.append(country_placeholder)
                                    
                                

# export it as dictionary
dict_author_edges_doi = {'doi_id': doi_ids, 'title': titles, 'keywords': keywords, 'abstract': abstracts, 'year_published': dates, 'first_author': first_author, 'last_author': last_author, 'affiliation_country' : affiliations_countries, 'affiliation': affiliations, 'affiliation_id': affiliations_ids} 
df_scholarly_doi = pd.DataFrame(dict_author_edges_doi)


In [32]:
#nr of publications from the lens dict
publications_doi = []
for scroll in data_dict_doi: 
    for pub in scroll: 
        publications_doi.append(pub['lens_id'])
len(set(publications_doi))

2435

In [33]:
#final amount after cleaning
len(set(df_scholarly_doi['doi_id']))

1843

In [34]:
df_scholarly_doi.to_csv(path+R'processed_data\scholarly_doi.csv')

In [35]:
#df_scholarly_pmid = pd.read_csv(path+R'processed_data\scholarly_pmid.csv', index_col = 0,keep_default_na=False, na_values=[np.nan])
#df_scholarly_pmcid = pd.read_csv(path+R'processed_data\scholarly_pmcid.csv', index_col = 0,keep_default_na=False, na_values=[np.nan])
#df_scholarly_doi = pd.read_csv(path+R'processed_data\scholarly_doi.csv', index_col = 0,keep_default_na=False, na_values=[np.nan])

In [36]:
df_scholarly = pd.DataFrame(pd.concat([df_scholarly_pmid['title'], df_scholarly_pmcid['title'], df_scholarly_doi['title']]).drop_duplicates(), columns = ['title']).reset_index(drop = True)

In [54]:
len(set(df_scholarly['title']))

96841

In [56]:
df_scholarly = pd.merge(df_scholarly, df_scholarly_pmid[['title', 'pubmed_id']].drop_duplicates(), how = 'left')
df_scholarly = pd.merge(df_scholarly, df_scholarly_pmcid[['title', 'pmcid_id']].drop_duplicates(), how = 'left')
df_scholarly = pd.merge(df_scholarly, df_scholarly_doi[['title', 'doi_id']].drop_duplicates(), how = 'left')


In [58]:
df_other_pub_info = pd.DataFrame(pd.concat([df_scholarly_pmid.drop(columns = ['pubmed_id', 'keywords']), df_scholarly_pmcid.drop(columns = ['pmcid_id', 'keywords']), df_scholarly_doi.drop(columns = ['doi_id', 'keywords'])])).drop_duplicates().reset_index(drop = True)

In [62]:
df_scholarly  = pd.merge(df_scholarly, df_other_pub_info, how = 'left')
#at this stage a a handful of publications has multiple ids because they were wrongly assigned the same title, but I will those few cases in the next step)

### 2. Select one id per publication for further analysis

In [64]:
def create_title_id_dict(df):
    # Make sure to select the relevant columns and handle duplicates
    df_nodup = df[['title', 'pubmed_id', 'pmcid_id', 'doi_id']].drop_duplicates().reset_index(drop = True)
    
    # Create a new Series with the first valid ID in the order of priority
    df_nodup['selected_id'] = df_nodup[['pubmed_id', 'pmcid_id', 'doi_id']].bfill(axis=1).iloc[:, 0]
    
    # Create a dictionary mapping titles to their selected IDs
    title_id_dict = df_nodup.set_index('title')['selected_id'].to_dict()
    
    return title_id_dict

In [66]:
# Create the title-to-ID dictionary
title_id_dict = create_title_id_dict(df_scholarly)

C:\Users\a494-nipw010\AppData\Local\Temp\ipykernel_20044\2160015082.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_nodup['selected_id'] = df_nodup[['pubmed_id', 'pmcid_id', 'doi_id']].bfill(axis=1).iloc[:, 0]


In [68]:
df_scholarly['id_select'] = df_scholarly['title'].map(title_id_dict)

In [70]:
#final nr of publications 
len(set(df_scholarly['id_select']))

96841

In [3]:
101111 - 96841

4270

In [72]:
df_scholarly.to_csv(path+R'processed_data\scholarly.csv')

### overview

In [74]:
df_scholarly[df_scholarly['title'].isna()]

,title,pubmed_id,pmcid_id,doi_id,abstract,year_published,first_author,last_author,affiliation_country,affiliation,affiliation_id,id_select


In [76]:
len(set(df_scholarly['id_select'][df_scholarly['abstract'] == '']))

19381

In [78]:
len(set(df_scholarly['id_select'][df_scholarly['year_published']== '']))

0

In [80]:
len(set(df_scholarly['id_select'][df_scholarly['first_author']== '']))

0

In [82]:
len(set(df_scholarly['id_select'][df_scholarly['last_author']== '']))

0

In [84]:
#nr of publications where at least one affiliation country is known
len(set(df_scholarly['id_select'][df_scholarly['affiliation_country'].notna()]))

84859

In [86]:
#nr of publiations where at least on affiliation country is unknown
len(set(df_scholarly['id_select'][df_scholarly['affiliation_country'].isna()]))

48976

In [88]:
#nr of publiations where at least on affiliation is known
len(set(df_scholarly['id_select'][df_scholarly['affiliation']!= '']))

95621

In [90]:
len(set(df_scholarly['id_select'][df_scholarly['affiliation']== '']))

4536

In [92]:
len(set(df_scholarly['id_select'][df_scholarly['id_select'] == '']))

0